In [1]:
#from faker import Faker
from datetime import datetime
import string
import random
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import *
import tensorflow as tf
from tensorflow import keras
import keras
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, Activation,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import sys
from tensorflow.keras.models import load_model

Using TensorFlow backend.


In [ ]:
#pd.set_option('display.max_rows', None)

In [2]:
dataset=pd.read_csv(r'datasetenglish.csv',dtype={'pin_code':str,'Aadhar':str})
dataset.drop('Unnamed: 0',axis=1,inplace=True)
dataset.drop('First_Name',axis=1,inplace=True)
dataset.drop('Last_Name',axis=1,inplace=True)
dataset

,row_id,Name,Address,DOB,Aadhar,PAN,Drive_License,Passport,location,city,pin_code
0,5165,"doshi, rigwaan",4 vafaadaar dharwar-014354,1976-09-14,153182030027,WRZBEXV8RB,DCC14943D7004,W78307214,NaN,NaN,NaN
1,3466,"bana, pradeep",0 kanti shrvimal chiraala-372797,1994-12-31,320418719806,EHH35RBP5X,C690E77DE25E4,M39491062,NaN,NaN,NaN
2,3998,rajeev mahadev,2 mandal lakhanu-957257,2012-07-10,653261738557,DYJZ11372T,72B5A03A77374,S84366105,NaN,NaN,NaN
3,776,sulabha luthra,1/4 aditya singh antargat 774739,2011-04-22,382030391659,YZHALL63CZ,7B77655809474,E11132807,NaN,NaN,NaN
4,8265,"lokanatyon, lalit",49 sai dhaliwal jisasebelari-897535,1995-04-01,747174064230,JDL9FFL53J,706DDFD3AB9A4,D80935986,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
12494,3731,jainti gupta,31 daal chandigarh 411748,1932-02-17,944484976011,XKCOA8VG61,F15017F6B9424,J54216262,NaN,NaN,NaN
12495,3137,ankitha kunda,28 choudhary dehara-732150,1916-02-29,555452479922,THRWTDNCLV,EA139F766F8B4,E49482722,NaN,NaN,NaN
12496,5745,akhil badami,924 kunda hyderabad-888760,1967-03-14,127606063264,CHK2Q7F1ON,7B5F44D1FC774,K21096755,NaN,NaN,NaN
12497,762,"bakshi, ayaansh",9472 chouhan aadilabad 556613,1963-12-04,741132642396,HPOF0DGPPW,F2BEA3F122A94,K15326012,NaN,NaN,NaN


In [3]:
# Name = Words  Example = doshi, rigwaan,, sulabha luthra
# Address = Word + Number  Example = 4 vafaadaar dharwar-014354 ,, 31 daal chandigarh 411748
# DOB = Year-Month-Date(Number)   Example = 1976-09-14 ,, 1995-04-01
# Aadhar = 12 digit number   Example = 153182030027 ,, 653261738557
# PAN = 10 Alphanumericals   Example = WRZBEXV8RB ,, DYJZ11372T
# Drive_License = 13 Alphanumericals   Example = DCC14943D7004 ,, 7B77655809474
# Passport = 1 Alphabet + 8 Numbers   Example = W78307214 ,, E49482722

In [3]:
dataset

,row_id,Name,Address,DOB,Aadhar,PAN,Drive_License,Passport,location,city,pin_code
0,5165,"doshi, rigwaan",4 vafaadaar dharwar-014354,1976-09-14,153182030027,WRZBEXV8RB,DCC14943D7004,W78307214,NaN,NaN,NaN
1,3466,"bana, pradeep",0 kanti shrvimal chiraala-372797,1994-12-31,320418719806,EHH35RBP5X,C690E77DE25E4,M39491062,NaN,NaN,NaN
2,3998,rajeev mahadev,2 mandal lakhanu-957257,2012-07-10,653261738557,DYJZ11372T,72B5A03A77374,S84366105,NaN,NaN,NaN
3,776,sulabha luthra,1/4 aditya singh antargat 774739,2011-04-22,382030391659,YZHALL63CZ,7B77655809474,E11132807,NaN,NaN,NaN
4,8265,"lokanatyon, lalit",49 sai dhaliwal jisasebelari-897535,1995-04-01,747174064230,JDL9FFL53J,706DDFD3AB9A4,D80935986,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
12494,3731,jainti gupta,31 daal chandigarh 411748,1932-02-17,944484976011,XKCOA8VG61,F15017F6B9424,J54216262,NaN,NaN,NaN
12495,3137,ankitha kunda,28 choudhary dehara-732150,1916-02-29,555452479922,THRWTDNCLV,EA139F766F8B4,E49482722,NaN,NaN,NaN
12496,5745,akhil badami,924 kunda hyderabad-888760,1967-03-14,127606063264,CHK2Q7F1ON,7B5F44D1FC774,K21096755,NaN,NaN,NaN
12497,762,"bakshi, ayaansh",9472 chouhan aadilabad 556613,1963-12-04,741132642396,HPOF0DGPPW,F2BEA3F122A94,K15326012,NaN,NaN,NaN


In [4]:
# Converting the dataset into input feature and label type
# 0-Name = Words  Example = doshi, rigwaan,, sulabha luthra
# 1-Address = Word + Number  Example = 4 vafaadaar dharwar-014354 ,, 31 daal chandigarh 411748
# 2-DOB = Year-Month-Date(Number)   Example = 1976-09-14 ,, 1995-04-01
# 3-Aadhar = 12 digit number   Example = 153182030027 ,, 653261738557
# 4-PAN = 10 Alphanumericals   Example = WRZBEXV8RB ,, DYJZ11372T
# 5-Drive_License = 13 Alphanumericals   Example = DCC14943D7004 ,, 7B77655809474
# 6-Passport = 1 Alphabet + 8 Numbers   Example = W78307214 ,, E49482722

dataset_medium=dataset[['Name','Address','DOB','Aadhar','PAN', 'Drive_License', 'Passport']]
dataset_medium
abc=dataset_medium
abc.shape
abc.dropna(inplace=True)
abc.shape
abc
new1=abc['Name']
new2=abc['Address']
new3=abc['DOB']
new4=abc['Aadhar']
new5=abc['PAN']
new6=abc['Drive_License']
new7=abc['Passport']
new1=new1.rename('Sample')
new1=new1.to_frame()
new1['Label']=0
new1
new2=new2.rename('Sample')
new2=new2.to_frame()
new2['Label']=1
new2
new3=new3.rename('Sample')
new3=new3.to_frame()
new3['Label']=2
new3
new4=new4.rename('Sample')
new4=new4.to_frame()
new4['Label']=3
new4
new5=new5.rename('Sample')
new5=new5.to_frame()
new5['Label']=4
new5
new6=new6.rename('Sample')
new6=new6.to_frame()
new6['Label']=5
new6
new7=new7.rename('Sample')
new7=new7.to_frame()
new7['Label']=6
new7
combine=pd.concat([new1,new2,new3,new4,new5,new6,new7])
combine.reset_index(inplace=True, drop=True)
combine
combine_csv = combine.to_csv(r'combine7.csv', header=True)

/Users/apple/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
combine=pd.read_csv(r'combine7.csv')
combine.drop('Unnamed: 0',axis=1,inplace=True)
combine

,Sample,Label
0,"doshi, rigwaan",0
1,"bana, pradeep",0
2,rajeev mahadev,0
3,sulabha luthra,0
4,"lokanatyon, lalit",0
...,...,...
69988,J54216262,6
69989,E49482722,6
69990,K21096755,6
69991,K15326012,6


In [6]:
X=combine['Sample']
X=X.to_frame()
X

,Sample
0,"doshi, rigwaan"
1,"bana, pradeep"
2,rajeev mahadev
3,sulabha luthra
4,"lokanatyon, lalit"
...,...
69988,J54216262
69989,E49482722
69990,K21096755
69991,K15326012


In [7]:
y=combine['Label']
y=y.to_frame()
y

,Label
0,0
1,0
2,0
3,0
4,0
...,...
69988,6
69989,6
69990,6
69991,6


In [8]:
# one hot encoding of data
data = y
data = np.array(data)
print(data)
encoded = to_categorical(data)
print(encoded)

[[0]
 [0]
 [0]
 ...
 [6]
 [6]
 [6]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [9]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, encoded, test_size=0.1, random_state=42)

In [ ]:
##########

In [10]:
# Tokenize the input at character level
max_features = 80000
tokenizer = Tokenizer(num_words=max_features,char_level=True,lower=False)#char_level=True
tokenizer.fit_on_texts(list(X_train['Sample']))
list_tokenized_train=tokenizer.texts_to_sequences(X_train['Sample'])
list_tokenized_test = tokenizer.texts_to_sequences(X_test['Sample'])
# Convert the input to numpy format
list_tokenized_train = np.asarray(list_tokenized_train)
list_tokenized_test = np.asarray(list_tokenized_test)
y_train = np.asarray(y_train)
# Pad the input data
maxlen = 10
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [11]:
# Embedding Size
embed_size = 50
# Input
inp=Input(shape=(maxlen, ))
# Embedding layer
x = Embedding(max_features, embed_size)(inp)
# LSTM Layer
x = LSTM(10, return_sequences=True,name='lstm_layer')(x)
# Maxpooling Layer
x = GlobalMaxPool1D()(x)
# Dense Layer
x = Dense(20, activation="relu")(x)
# Dropout Layer
x = Dropout(0.1)(x)
# Output Layer
x = Dense(7, activation="sigmoid")(x)

In [12]:
# Load the model
model = Model(inputs=inp, outputs=x)

In [13]:
# Compile the model
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [14]:
# Train the model
batch_size = 128
epochs = 3
model.fit(X_t,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 50394 samples, validate on 12599 samples
Epoch 1/3
50394/50394 [==============================] - 53s 1ms/sample - loss: 0.3395 - accuracy: 0.8765 - val_loss: 0.1117 - val_accuracy: 0.9665
Epoch 2/3
50394/50394 [==============================] - 48s 959us/sample - loss: 0.0571 - accuracy: 0.9881 - val_loss: 0.0132 - val_accuracy: 0.9991
Epoch 3/3
50394/50394 [==============================] - 54s 1ms/sample - loss: 0.0166 - accuracy: 0.9976 - val_loss: 0.0058 - val_accuracy: 0.9993


In [16]:
# Use the trained model for making a prediction.
z=['409402251']
zz=tokenizer.texts_to_sequences(z)
zzz = pad_sequences(zz, maxlen=maxlen)
xy=model.predict(zzz)
s=np.argmax(xy,axis=1)
if s==0:
    print('It is a Name')
elif s==1:
    print('It is a Address')
elif s==2:
    print('It is a DOB')
elif s==3:
    print('It is a Aadhar')
elif s==4:
    print('It is a PAN')
elif s==5:
    print('It is a Drive_License')
elif s==6:
    print('It is a Passport')

It is a Aadhar
